# Deck

## Requirement

- suits not required
- multiple decks
- need to be able to shuffle the deck
- need to draw a card from the deck

## Plan

- class: deck
    - can take number of decks as a variable
    - default 1 deck
- method: shuffle
- method: deal
- attribute: list of all the cards in the 'deck'

## Code

In [16]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [19]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']


# Calculate points

## Requirement

- Take a hand
- Calculate how many points there are in the hand
    - J, Q, K are 10
    - Ace could be 1 or 11
        - How do we decide?
        - If 11 doesn't bust, then 11, else 1
            - We have to calculate the rest of the cards first

## Plan

- Remove Aces in the hand
    - Track how many Aces there were
- Count J, Q, K as 10

## Code

In [35]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [40]:
calculate_points([5, 5, "A"])

21

# Simulate game